In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import cross_val_score,train_test_split

import sys
sys.path.append("..")

In [ ]:
df=pd.read_csv("/Users/maceli/Documents/Business case with data science/07 - Case Products recommendation/BigBasketDataset.tsv", sep='\t')

<b style='color: #2c786c;font-size:19px'>Table of contents</b>
<ul>
    <li style='color: #2c786c'><a href="#0">Exploring the database</a></li>
    <li style='color: #2c786c'><a href="#1">Take a look </a></li>
    <li style='color: #2c786c'><a href="#2">Clusters </a></li>
    <li style='color: #2c786c'><a href="#3">Item - Item Collaborative Filter Recommender </a></li>
    
</ul>

# Count member 

<b style='color: #2c786c; font-size: 17px' id="0"> Exploring the database </b>

In [ ]:
print("There are {} orders".format(*df.shape))

In [ ]:
member=pd.DataFrame({'count' : df.groupby( [ "Member"] ).size()}).reset_index()
print("There are {} distinct members".format(*member.shape))

In [ ]:
Description=pd.DataFrame({'count' : df.groupby( [ "Description"] ).size()}).reset_index()
print("There are {} distinct Descriptions".format(*Description.shape))

In [ ]:
SKU=pd.DataFrame({'count' : df.groupby( [ "SKU"] ).size()}).reset_index()
print("There are {} distinct SKU".format(*SKU.shape))

In [ ]:
DescriptionSKU=pd.DataFrame({'SKU' : df.groupby( [ "Description"] ).size()}).reset_index()
print("There are {} distinct SKU".format(*DescriptionSKU.shape))

In [ ]:
SKU_Description=pd.DataFrame({'count' : df.groupby( [ "SKU","Description"] ).size()}).reset_index()
print("There are {} combination SKU x Descriptions ".format(*SKU_Description.shape))

In [ ]:
df['Created On'] =  pd.to_datetime(df['Created On'])
df['day']      =df['Created On'].dt.dayofweek
df['day_month']=df['Created On'].dt.day
df['week']   =df['Created On'].dt.weekofyear
df['month']=df['Created On'].dt.month
df['year']=df['Created On'].dt.year

<b style='color: #2c786c; font-size: 17px' id="1"> Take a look </b>

In [ ]:
Agarbatti=df[df.Description=='Agarbatti']
Agarbatti.head(2)

In [ ]:
print("There are {} different SKU ".format(Agarbatti.SKU.unique()))

In [ ]:
order=df[df.Order==6445802]
print("There are {} different year ".format(Agarbatti.year.unique()))

In [ ]:
Member=df[df.Member=='M77779']

In [ ]:
Member.Order.unique()

In [ ]:
Memberdf = Member.groupby('Member')['Order'].nunique()

In [ ]:
Memberdf

### Coment
Decided work with Description, because for different SKU there are the same description,
Its could be cheack in cell above

In [ ]:
member_order2=pd.DataFrame({'count' : df.groupby( ['Member','Order'] ).size()}).reset_index()
member_order2.head()

In [ ]:
df_2=df.drop(['SKU'],axis=1)
#df_2.year.unique() 

In [ ]:
Description_order=pd.DataFrame({'count' : df.groupby( ['Description'] ).size()}).reset_index().sort_values('count', ascending=False)
Description_order.head()

In [ ]:
Description_order[Description_order.Description.str.contains("Other")]

In [120]:
from datetime import date
from datetime import datetime
today = datetime.now()
print("Today's date:", today)

Today's date: 2020-05-24 01:02:54.591402


In [ ]:
df_2['Created On'] =  pd.to_datetime(df_2['Created On'], format='%Y/%m/%d')

In [121]:
df_2.head()

,Member,Order,SKU,Created On,Description,day,day_month,week,month,year
1,M09736,6468572,15669800,2014-09-22 22:45:00,Cashews,0,22,39,9,2014
3,M09736,6468572,7572303,2014-09-22 22:45:00,Namkeen,0,22,39,9,2014
4,M09736,6468572,15669856,2014-09-22 22:45:00,Sugar,0,22,39,9,2014
5,M09736,6468572,15668478,2014-09-22 22:45:00,Banana,0,22,39,9,2014
6,M09736,6468572,21409124,2014-09-22 22:45:00,Sugar Cubes,0,22,39,9,2014


In [123]:
df_2[hhh] = (today-df_2['Created On1']).day()

KeyError: 'Created On1'

In [ ]:
quantidade_dias = abs((today - df_2['Created On']).days)

In [ ]:
df_2=df[~df.Description.str.contains("Other", na=False)]

In [ ]:
df_2.head()

In [ ]:
df_4=pd.DataFrame({'count' : df_2.groupby( ['Description'] ).size()}).reset_index()
df_4= df_4.sort_values('count', ascending=False)
dumies=pd.get_dummies(df_4['Description'], drop_first=True)
import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))

plt.barh(df_4['Description'].head(20),df_4['count'].head(20), align='center',
        color='skyblue')
plt.gca().invert_yaxis()
plt.xlabel("Popularity")
plt.title("Popular Produtos")

In [ ]:
member_order=pd.DataFrame({'count' : df_2.groupby( ['Member','Description'] ).size()}).reset_index()

In [ ]:
member_order.head()

In [ ]:
df_4= df_4.sort_values('count', ascending=False)
df_4

In [ ]:
import seaborn as sns
sns.countplot(x = 'month', data = df_2,hue='year')
              

In [ ]:
member_order=pd.DataFrame({'count' : df_2.groupby( ['month','Description'] ).size()}).reset_index()
member_order= member_order.sort_values(by=['month','count'], ascending=False)
member_order

In [ ]:
monht_top_orders=list()
for i in member_order.month.unique():
    monht_top_orders.append(member_order[member_order.month ==i][:10])
    print(monht_top_orders)


In [ ]:
sub_df2 = pd.DataFrame(monht_top_orders)

In [ ]:
sub_df2.head()

<b style='color: #2c786c; font-size: 17px' id="2"> Cluster </b>

In [ ]:
member_order=pd.DataFrame({'count' : df_2.groupby( ['Member','Description'] ).size()}).reset_index()
member_order_matrix =pd.DataFrame( pd.pivot_table(member_order, values='count', index='Member', columns='Description', fill_value=0))
member_order_matrix.rename_axis(None, axis=1)
member_order_matrix=member_order_matrix.rename_axis(None, axis=1).reset_index()
member_order_matrix_2=member_order_matrix[dumies.columns]
member_order_matrix_2.head()

In [ ]:
member_order=pd.DataFrame({'count' : df_2.groupby( ['Member','Description'] ).size()}).reset_index()
member_order_matrix =pd.DataFrame( pd.pivot_table(member_order, values='count', index='Member', columns='Description', fill_value=0))

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
mat_movie_features = csr_matrix(member_order_matrix.values)

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns
sse = {}
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=1)
    kmeans.fit(member_order_matrix_2)
    sse[k] = kmeans.inertia_
# Plot SSE for each *k*
plt.title('The Elbow Method')
plt.xlabel('k'); plt.ylabel('SSE')
sns.pointplot(x=list(sse.keys()), y=list(sse.values()))
plt.show()

plt.savefig('plot.png')

In [ ]:
km_clf = KMeans(verbose = 0,n_clusters = 8
               )
km_clf.fit(member_order_matrix_2)

In [ ]:
member_order_matrix['Cluster_Id'] = km_clf.labels_

In [ ]:
member_order_matrix.head()

In [ ]:
filter=['Member','Cluster_Id']
member_order_matrix=member_order_matrix.rename_axis(None, axis=1).reset_index()
member_order_matrix=member_order_matrix[filter]

In [ ]:
member_cluster=pd.DataFrame({'count' : member_order_matrix.groupby( ['Cluster_Id'] ).size()}).reset_index()

In [ ]:
member_order_cluster=member_order.set_index('Member').join(member_order_matrix.set_index('Member'))

In [ ]:
member_order_matrix.head()

In [ ]:
member_order_cluster.head()

In [ ]:
range(km_clf.n_clusters)

In [ ]:
cluster_top_features = list()
# Sort each centroid vector to find the top 20 features
for i in range(km_clf.n_clusters):
      cluster_top_features.append(np.argsort(km_clf.cluster_centers_[i])[::-1][:10])
# Map vector back to words and print out

for num, centroid in enumerate(cluster_top_features):
    print ('Cluster',num,':',(", ".join(dumies.columns[i] for i in centroid)))

<b style='color: #2c786c; font-size: 17px' id="3"> Item - Item Collaborative Filter Recommender </b>

In [ ]:
print(member_order_matrix.groupby('Member').count()) 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from time import time
class ItemItemRecommender(object):

    def __init__(self, neighborhood_size):
        self.neighborhood_size = neighborhood_size

    def fit(self, ratings_mat):
        self.ratings_mat = ratings_mat
        self.n_users = ratings_mat.shape[0]
        self.n_items = ratings_mat.shape[1]
        self.item_sim_mat = cosine_similarity(self.ratings_mat.T)  ##########
        self._set_neighborhoods()

    def _set_neighborhoods(self):
        least_to_most_sim_indexes = np.argsort(self.item_sim_mat, 1)
        self.neighborhoods = least_to_most_sim_indexes[:, -self.neighborhood_size:]

    def pred_one_user(self, user_id, report_run_time=False):
        start_time = time()
        items_rated_by_this_user = self.ratings_mat[user_id].nonzero()[1]
        # Just initializing so I have somewhere to put rating preds
        out = np.zeros(self.n_items)
        for item_to_rate in range(self.n_items):
            relevant_items = np.intersect1d(self.neighborhoods[item_to_rate],
                                            items_rated_by_this_user,  ##### = self.ratings_mat[user_id].nonzero()[1]
                                            assume_unique=True)  #########   assume_unique speeds up intersection op
            out[item_to_rate] = self.ratings_mat[user_id, relevant_items] * \
                self.item_sim_mat[item_to_rate, relevant_items] / \
                self.item_sim_mat[item_to_rate, relevant_items].sum()
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        cleaned_out = np.nan_to_num(out)
        return cleaned_out

    def pred_all_users(self, report_run_time=False):
        start_time = time()
        all_ratings = [
            self.pred_one_user(user_id) for user_id in range(self.n_users)]  #ratings_mat.shape[0]
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        return np.array(all_ratings)

    def top_n_recs(self, user_id, n):
        pred_ratings = self.pred_one_user(user_id)
        item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))
        items_rated_by_this_user = self.ratings_mat[user_id].nonzero()[1]
        unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                        if item not in items_rated_by_this_user]
        return unrated_items_by_pred_rating[-n:], items_rated_by_this_user

In [ ]:
active_user2 = list(reviews_count_df[reviews_count_df >= 100].index)  
mask2 = [user in active_user2 for user in recommender_df['user_id']]
active_user_df2 = recommender_df[mask2]
active_user2_ls = list(active_user_df2['user_id'].unique())
print(active_user2_ls)

In [ ]:
member_order_matrix.head()

In [ ]:
class ItemItemRecommender(object):

    def __init__(self, neighborhood_size):
        self.neighborhood_size = neighborhood_size

    def fit(self, ratings_mat):
        self.ratings_mat = ratings_mat
        self.n_users = ratings_mat.shape[0]
        self.n_items = ratings_mat.shape[1]
        self.item_sim_mat = cosine_similarity(self.ratings_mat.T)  ##########
        self._set_neighborhoods()

In [ ]:
mat_movie_features = csr_matrix(member_order_matrix_2.values)

In [ ]:
my_rec_engine = ItemItemRecommender(neighborhood_size=80)
my_rec_engine.fit(mat_movie_features) 

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
from collections import bCounter

In [ ]:
member_order=pd.DataFrame({'count' : df_2.groupby( ['Member','Description'] ).size()}).reset_index()
member_order= member_order.sort_values(by=['Member','count'], ascending=False)
member_order

In [ ]:
from scipy import sparse
highest_user_id = len(set(member_order['Member']))
highest_movie_id = len(set(member_order['business_id']))
ratings_mat = sparse.lil_matrix((highest_user_id, highest_movie_id))
ratings_mat

In [ ]:
user_id = list(set(member_order['Member']))

In [ ]:
user_id

In [ ]:
user_id = list(set(member_order['Member']))
business_id = list(set(member_order['Description']))
for _, row in member_order.iterrows():
    print(row)
    #ratings_mat[user_id.index(row.Member), business_id.index(row.Description)] = row.count  #fill the new_clusterin ratings_mat based on the active_user_df
ratings_mat

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from time import time
class ItemItemRecommender(object):

    def __init__(self, neighborhood_size):
        self.neighborhood_size = neighborhood_size

    def fit(self, ratings_mat):
        self.ratings_mat = ratings_mat
        self.n_users = ratings_mat.shape[0]
        self.n_items = ratings_mat.shape[1]
        self.item_sim_mat = cosine_similarity(self.ratings_mat.T)  ##########
        self._set_neighborhoods()

    def _set_neighborhoods(self):
        least_to_most_sim_indexes = np.argsort(self.item_sim_mat, 1)
        self.neighborhoods = least_to_most_sim_indexes[:, -self.neighborhood_size:]

    def pred_one_user(self, user_id, report_run_time=False):
        start_time = time()
        items_rated_by_this_user = self.ratings_mat[user_id].nonzero()[1]
        # Just initializing so I have somewhere to put rating preds
        out = np.zeros(self.n_items)
        for item_to_rate in range(self.n_items):
            relevant_items = np.intersect1d(self.neighborhoods[item_to_rate],
                                            items_rated_by_this_user,  ##### = self.ratings_mat[user_id].nonzero()[1]
                                            assume_unique=True)  #########   assume_unique speeds up intersection op
            out[item_to_rate] = self.ratings_mat[user_id, relevant_items] * \
                self.item_sim_mat[item_to_rate, relevant_items] / \
                self.item_sim_mat[item_to_rate, relevant_items].sum()
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        cleaned_out = np.nan_to_num(out)
        return cleaned_out

    def pred_all_users(self, report_run_time=False):
        start_time = time()
        all_ratings = [
            self.pred_one_user(user_id) for user_id in range(self.n_users)]  #ratings_mat.shape[0]
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        return np.array(all_ratings)

    def top_n_recs(self, user_id, n):
        pred_ratings = self.pred_one_user(user_id)
        item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))
        items_rated_by_this_user = self.ratings_mat[user_id].nonzero()[1]
        unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                        if item not in items_rated_by_this_user]
        return unrated_items_by_pred_rating[-n:], items_rated_by_this_user

In [ ]:
my_rec_engine = ItemItemRecommender(neighborhood_size=80)
my_rec_engine.fit(ratings_mat) 

In [ ]:
ratings_mat

In [ ]:
lucky_user = np.random.choice(member_order['Member'])
lucky_user_index = user_id.index(lucky_user)
lucky_user_recommend, items_rated_by_this_user = my_rec_engine.top_n_recs(user_id=53, n = 10)

In [ ]:
items_rated_by_this_user = my_rec_engine.top_n_recs(user_id=53, n = 10)

In [ ]:
lucky_user_recommend

In [ ]:
print("The top ten recommendation for user %s are: " % (lucky_user))

In [ ]:
member_order[member_order.Member=='M22037']

In [ ]:
print('%s' % (', '.join(list(set(df['Description'][df['Description'] == business_id[i]]))[0] \
                       for i in lucky_user_recommend)))